In [1]:
import joblib
import os
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
os.listdir('./model')

['ANN(58_to_sp162).pkl',
 'ANN(energy).pkl',
 'SVR(5_to_23).pkl',
 'transformer(23_to_54).pkl',
 'transformer(SP162_to_Y162).pkl']

In [2]:
os.listdir('./data/phase_2/cleaned')

['col_names.pkl', 'train(energy).csv', 'train.csv']

# load pretrained model

In [3]:
from API.utils import *
A = joblib.load('./model/SVR(5_to_23).pkl')
B = joblib.load('./model/transformer(23_to_54).pkl')
C = joblib.load('./model/ANN(58_to_sp162).pkl')
D = joblib.load('./model/transformer(SP162_to_Y162).pkl')
E = joblib.load('./model/ANN(energy).pkl')
col_names = joblib.load('./data/phase_2/cleaned/col_names.pkl')

# step1

In [35]:
df = pd.read_csv('./data/phase_1/test_170.csv',index_col=0).sample(1)
y_test = df.iloc[:,4:]
X_test = df[['T10','T50','T90','N+A']]
X_test

,T10,T50,T90,N+A
145,87.1,110.5,143.7,46.739


In [36]:
y_pred23 = A.predict(X_test)

In [37]:
y_pred23 = pd.DataFrame(y_pred23,columns=y_test.columns)
y_pred23

,C5NP,C5IP,C5N,C6NP,C6IP,C6N,C6A,C7NP,C7IP,C7N,C7A,C8NP,C8IP,C8N,C8A,C9NP,C9IP,C9N,C9A,C10NP,C10IP,C10N,C10A
0,3.257333,1.914251,0.364495,3.987746,3.870913,5.793746,0.783988,6.320215,6.701499,14.953893,3.519006,5.590371,6.469113,8.157472,4.557477,3.02019,7.326849,4.720786,2.221224,0.839696,3.962824,0.764537,0.902376


In [29]:
print(y_pred23.sum(axis=1))
t = y_pred23.append(y_test)
t.index = ['pred','real']
t.loc['error'] = abs(t.T['pred'] - t.T['real'].T)
t['sum'] = t.sum(axis=1)
t

0    100.0
dtype: float64


,C5NP,C5IP,C5N,C6NP,C6IP,C6N,C6A,C7NP,C7IP,C7N,C7A,C8NP,C8IP,C8N,C8A,C9NP,C9IP,C9N,C9A,C10NP,C10IP,C10N,C10A,sum
pred,0.059201,0.068786,0.099906,3.668041,1.139182,2.206815,0.430575,11.254262,8.368803,6.361806,2.97153,10.136475,10.768359,5.006557,5.487379,6.732708,10.840554,4.502236,3.328659,0.79673,4.972899,0.477959,0.320576,100.0000
real,0.055000,0.029000,0.013000,3.627000,1.091000,2.237000,0.367000,10.672000,7.610000,6.347000,2.84000,9.815000,10.466000,4.976000,5.426000,6.880000,11.145000,4.219000,3.871000,0.95900,5.933000,0.519000,0.379000,99.4760
error,0.004201,0.039786,0.086906,0.041041,0.048182,0.030185,0.063575,0.582262,0.758803,0.014806,0.13153,0.321475,0.302359,0.030557,0.061379,0.147292,0.304446,0.283236,0.542341,0.16227,0.960101,0.041041,0.058424,5.0162


# step 2

In [7]:
y_pred54 = B(y_pred23)

In [8]:
print(y_pred54.sum(axis=1))
y_pred54.head()

0    100.0
dtype: float64


,O2,METHANE,ETHANE,PROPANE,BUTANE,IPENTANE,PENTANE,T2PENTEN,CP,3MP,T2HEXENE,HEXANE,MCP,22MP,BENZENE,CH,223MB,4M1HEXEN,2MHX,1T3M,HEPTANE,H2O,MCH,ECP,23DM1HX,TOLUENE,2MHEPT,OCTANE,ECH,EBENZENE,PXYLENE,MXYLENE,OXYLENE,2MOCTN,1NONENE,NONANE,CUMENE,1T35TMCH,PCH,PRBENZEN,135MBENZ,1M2EBNZN,123MBENZ,24DMOCT,TEMH,3MN,TBCYHXAN,DECANE,ODEBZ,UNDECANE,PNBZ,DODECANE,HXBNZN,TRIDECAN
0,0.0,0.0,0.0,0.003439,0.006879,0.057093,0.059201,0.001376,0.099906,1.116398,0.022784,3.668041,1.431686,0.801621,0.430575,0.775129,0.880815,0.01757,6.668797,0.588884,11.254262,0.0,3.411197,2.361726,0.107684,2.97153,10.660676,10.136475,5.006557,1.303005,0.739147,1.824402,1.620825,10.565175,0.275379,6.732708,0.047178,2.701218,1.801018,0.201235,0.864194,1.277181,0.938871,0.83626,1.240801,2.895837,0.477959,0.668115,0.29977,0.126362,0.017428,0.002253,0.003378,0.0


# step3

In [9]:
df = pd.read_csv('./data/phase_2/cleaned/train.csv',index_col=0)

In [10]:
idx = []
for i in df.index:
    if i.split('-')[0] in ['017','113','142','148']:
        idx.append(i)

In [11]:
df = df.loc[idx]
x = df[col_names['case']+col_names['xna']] #58
sp = df[col_names['sle']+col_names['shc']+col_names['she']] #162
y = df[col_names['xle']+col_names['xhc']+col_names['xhe']] #162
df

,Case Conditions_Feed Rate to C013 (Input)_m3/hr,Case Conditions_Heart Cut Prod. Rate (Input)_m3/hr,Case Conditions_C7+ in Light End (Input)_wt%,Case Conditions_C6P- in Heart Cut (Input)_wt%,Naphtha Properties_Oxygen_wt%,Naphtha Properties_Methane_wt%,Naphtha Properties_Ethane_wt%,Naphtha Properties_Propane_wt%,Naphtha Properties_n-Butane_wt%,Naphtha Properties_i-Pentane_wt%,Naphtha Properties_n-Pentane_wt%,Naphtha Properties_tr2-Pentene_wt%,Naphtha Properties_Cyclopentane_wt%,Naphtha Properties_3-Methylpentane_wt%,Naphtha Properties_tr2-Hexene_wt%,Naphtha Properties_n-Hexane_wt%,Naphtha Properties_Methylcyclopentane_wt%,"Naphtha Properties_2,2-Dimethylpentane_wt%",Naphtha Properties_Benzene_wt%,Naphtha Properties_Cyclohexane_wt%,"Naphtha Properties_2,2,3-Trimethylbutane_wt%",Naphtha Properties_4-Methyl-1-hexene_wt%,Naphtha Properties_2-Methylhexane_wt%,"Naphtha Properties_t-1,3-Dimethylcyclopentane_wt%",Naphtha Properties_n-Heptane_wt%,Naphtha Properties_H2O_wt%,Naphtha Properties_Methylcyclohexane_wt%,Naphtha Properties_Ethylcyclopentane_wt%,"Naphtha Properties_2,3-Dimethyl-1-hexene_wt%",Naphtha Properties_Toluene_wt%,Naphtha Properties_2-Methylheptane_wt%,Naphtha Properties_n-Octane_wt%,Naphtha Properties_Ethylcyclohexane_wt%,Naphtha Properties_EthylBenzene_wt%,Naphtha Properties_P-Xylene_wt%,Naphtha Properties_M-Xylene_wt%,Naphtha Properties_O-Xylene_wt%,Naphtha Properties_2-Methyloctane_wt%,Naphtha Properties_1-Nonene_wt%,Naphtha Properties_n-Nonane_wt%,Naphtha Properties_i-Propylbenzene_wt%,"Naphtha Properties_1-trans-3,5-Trimethylcyclohexane_wt%",Naphtha Properties_n-Propylcyclohexane_wt%,Naphtha Properties_n-Propylbenzene_wt%,"Naphtha Properties_1,3,5-Trimethylbenzene_wt%",Naphtha Properties_1-Methyl-2-ethylbenzene_wt%,"Naphtha Properties_1,2,3-Trimethylbenzene_wt%","Naphtha Properties_2,4-Dimethyloctane_wt%","Naphtha Properties_2,2,3,3-Tetramethylhexane_wt%",Naphtha Properties_3-Methylnonane_wt%,Naphtha Properties_tert-Butylcyclohexane_wt%,Naphtha Properties_n-Decane_wt%,"Naphtha Properties_1,2-Diethylbenzene_wt%",Naphtha Properties_n-Undecane_wt%,Naphtha Properties_n-Pentylbenzene_wt%,Naphtha Properties_n-Dodecane_wt%,Naphtha Properties_n-Hexylbenzene_wt%,Naphtha Properties_n-Tridecane_wt%,Light End Product Properties_Oxygen_wt%,Light End Product Properties_Methane_wt%,Light End Product Properties_Ethane_wt%,Light End Product Properties_Propane_wt%,Light End Product Properties_n-Butane_wt%,Light End Product Properties_i-Pentane_wt%,Light End Product Properties_n-Pentane_wt%,Light End Product Properties_tr2-Pentene_wt%,Light End Product Properties_Cyclopentane_wt%,Light End Product Properties_3-Methylpentane_wt%,Light End Product Properties_tr2-Hexene_wt%,Light End Product Properties_n-Hexane_wt%,Light End Product Properties_Methylcyclopentane_wt%,"Light End Product Properties_2,2-Dimethylpentane_wt%",Light End Product Properties_Benzene_wt%,Light End Product Properties_Cyclohexane_wt%,"Light End Product Properties_2,2,3-Trimethylbutane_wt%",Light End Product Properties_4-Methyl-1-hexene_wt%,Light End Product Properties_2-Methylhexane_wt%,"Light End Product Properties_t-1,3-Dimethylcyclopentane_wt%",Light End Product Properties_n-Heptane_wt%,Light End Product Properties_H2O_wt%,Light End Product Properties_Methylcyclohexane_wt%,Light End Product Properties_Ethylcyclopentane_wt%,"Light End Product Properties_2,3-Dimethyl-1-hexene_wt%",Light End Product Properties_Toluene_wt%,Light End Product Properties_2-Methylheptane_wt%,Light End Product Properties_n-Octane_wt%,Light End Product Properties_Ethylcyclohexane_wt%,Light End Product Properties_EthylBenzene_wt%,Light End Product Properties_P-Xylene_wt%,Light End Product Properties_M-Xylene_wt%,Light End Product Properties_O-Xylene_wt%,Light End Product Properties_2-Methyloctane_wt%,Light End Product Properties_1-Nonene_wt%,Light End Product Properties_n-Nonane_wt%,Light End Product Properties_i-Propylbenzene_wt%,"Light End Product Properties_1-trans-3,5-Trimethylcyclohexa

In [12]:
sp

,Individual Component to Light End Split Factor_Oxygen_Ratio,Individual Component to Light End Split Factor_Methane_Ratio,Individual Component to Light End Split Factor_Ethane_Ratio,Individual Component to Light End Split Factor_Propane_Ratio,Individual Component to Light End Split Factor_n-Butane_Ratio,Individual Component to Light End Split Factor_i-Pentane_Ratio,Individual Component to Light End Split Factor_n-Pentane_Ratio,Individual Component to Light End Split Factor_tr2-Pentene_Ratio,Individual Component to Light End Split Factor_Cyclopentane_Ratio,Individual Component to Light End Split Factor_3-Methylpentane_Ratio,Individual Component to Light End Split Factor_tr2-Hexene_Ratio,Individual Component to Light End Split Factor_n-Hexane_Ratio,Individual Component to Light End Split Factor_Methylcyclopentane_Ratio,"Individual Component to Light End Split Factor_2,2-Dimethylpentane_Ratio",Individual Component to Light End Split Factor_Benzene_Ratio,Individual Component to Light End Split Factor_Cyclohexane_Ratio,"Individual Component to Light End Split Factor_2,2,3-Trimethylbutane_Ratio",Individual Component to Light End Split Factor_4-Methyl-1-hexene_Ratio,Individual Component to Light End Split Factor_2-Methylhexane_Ratio,"Individual Component to Light End Split Factor_t-1,3-Dimethylcyclopentane_Ratio",Individual Component to Light End Split Factor_n-Heptane_Ratio,Individual Component to Light End Split Factor_H2O_Ratio,Individual Component to Light End Split Factor_Methylcyclohexane_Ratio,Individual Component to Light End Split Factor_Ethylcyclopentane_Ratio,"Individual Component to Light End Split Factor_2,3-Dimethyl-1-hexene_Ratio",Individual Component to Light End Split Factor_Toluene_Ratio,Individual Component to Light End Split Factor_2-Methylheptane_Ratio,Individual Component to Light End Split Factor_n-Octane_Ratio,Individual Component to Light End Split Factor_Ethylcyclohexane_Ratio,Individual Component to Light End Split Factor_EthylBenzene_Ratio,Individual Component to Light End Split Factor_P-Xylene_Ratio,Individual Component to Light End Split Factor_M-Xylene_Ratio,Individual Component to Light End Split Factor_O-Xylene_Ratio,Individual Component to Light End Split Factor_2-Methyloctane_Ratio,Individual Component to Light End Split Factor_1-Nonene_Ratio,Individual Component to Light End Split Factor_n-Nonane_Ratio,Individual Component to Light End Split Factor_i-Propylbenzene_Ratio,"Individual Component to Light End Split Factor_1-trans-3,5-Trimethylcyclohexane_Ratio",Individual Component to Light End Split Factor_n-Propylcyclohexane_Ratio,Individual Component to Light End Split Factor_n-Propylbenzene_Ratio,"Individual Component to Light End Split Factor_1,3,5-Trimethylbenzene_Ratio",Individual Component to Light End Split Factor_1-Methyl-2-ethylbenzene_Ratio,"Individual Component to Light End Split Factor_1,2,3-Trimethylbenzene_Ratio","Individual Component to Light End Split Factor_2,4-Dimethyloctane_Ratio","Individual Component to Light End Split Factor_2,2,3,3-Tetramethylhexane_Ratio",Individual Component to Light End Split Factor_3-Methylnonane_Ratio,Individual Component to Light End Split Factor_tert-Butylcyclohexane_Ratio,Individual Component to Light End Split Factor_n-Decane_Ratio,"Individual Component to Light End Split Factor_1,2-Diethylbenzene_Ratio",Individual Component to Light End Split Factor_n-Undecane_Ratio,Individual Component to Light End Split Factor_n-Pentylbenzene_Ratio,Individual Component to Light End Split Factor_n-Dodecane_Ratio,Individual Component to Light End Split Factor_n-Hexylbenzene_Ratio,Individual Component to Light End Split Factor_n-Tridecane_Ratio,Individual Component to Heart Cut Split Factor_Oxygen_Ratio,Individual Component to Heart Cut Split Factor_Methane_Ratio,Individual Component to Heart Cut Split Factor_Ethane_Ratio,Individual Component to Heart Cut Split Factor_Propane_Ratio,Individual Component to Heart Cut Split Factor_n-Butane_Ratio,Individual Component to Hea

In [13]:
sp_pred = C.predict(x)

In [14]:
sp_pred = sp_pred[sp.columns.tolist()]
sp_pred.index = idx
sp_pred

,Individual Component to Light End Split Factor_Oxygen_Ratio,Individual Component to Light End Split Factor_Methane_Ratio,Individual Component to Light End Split Factor_Ethane_Ratio,Individual Component to Light End Split Factor_Propane_Ratio,Individual Component to Light End Split Factor_n-Butane_Ratio,Individual Component to Light End Split Factor_i-Pentane_Ratio,Individual Component to Light End Split Factor_n-Pentane_Ratio,Individual Component to Light End Split Factor_tr2-Pentene_Ratio,Individual Component to Light End Split Factor_Cyclopentane_Ratio,Individual Component to Light End Split Factor_3-Methylpentane_Ratio,Individual Component to Light End Split Factor_tr2-Hexene_Ratio,Individual Component to Light End Split Factor_n-Hexane_Ratio,Individual Component to Light End Split Factor_Methylcyclopentane_Ratio,"Individual Component to Light End Split Factor_2,2-Dimethylpentane_Ratio",Individual Component to Light End Split Factor_Benzene_Ratio,Individual Component to Light End Split Factor_Cyclohexane_Ratio,"Individual Component to Light End Split Factor_2,2,3-Trimethylbutane_Ratio",Individual Component to Light End Split Factor_4-Methyl-1-hexene_Ratio,Individual Component to Light End Split Factor_2-Methylhexane_Ratio,"Individual Component to Light End Split Factor_t-1,3-Dimethylcyclopentane_Ratio",Individual Component to Light End Split Factor_n-Heptane_Ratio,Individual Component to Light End Split Factor_H2O_Ratio,Individual Component to Light End Split Factor_Methylcyclohexane_Ratio,Individual Component to Light End Split Factor_Ethylcyclopentane_Ratio,"Individual Component to Light End Split Factor_2,3-Dimethyl-1-hexene_Ratio",Individual Component to Light End Split Factor_Toluene_Ratio,Individual Component to Light End Split Factor_2-Methylheptane_Ratio,Individual Component to Light End Split Factor_n-Octane_Ratio,Individual Component to Light End Split Factor_Ethylcyclohexane_Ratio,Individual Component to Light End Split Factor_EthylBenzene_Ratio,Individual Component to Light End Split Factor_P-Xylene_Ratio,Individual Component to Light End Split Factor_M-Xylene_Ratio,Individual Component to Light End Split Factor_O-Xylene_Ratio,Individual Component to Light End Split Factor_2-Methyloctane_Ratio,Individual Component to Light End Split Factor_1-Nonene_Ratio,Individual Component to Light End Split Factor_n-Nonane_Ratio,Individual Component to Light End Split Factor_i-Propylbenzene_Ratio,"Individual Component to Light End Split Factor_1-trans-3,5-Trimethylcyclohexane_Ratio",Individual Component to Light End Split Factor_n-Propylcyclohexane_Ratio,Individual Component to Light End Split Factor_n-Propylbenzene_Ratio,"Individual Component to Light End Split Factor_1,3,5-Trimethylbenzene_Ratio",Individual Component to Light End Split Factor_1-Methyl-2-ethylbenzene_Ratio,"Individual Component to Light End Split Factor_1,2,3-Trimethylbenzene_Ratio","Individual Component to Light End Split Factor_2,4-Dimethyloctane_Ratio","Individual Component to Light End Split Factor_2,2,3,3-Tetramethylhexane_Ratio",Individual Component to Light End Split Factor_3-Methylnonane_Ratio,Individual Component to Light End Split Factor_tert-Butylcyclohexane_Ratio,Individual Component to Light End Split Factor_n-Decane_Ratio,"Individual Component to Light End Split Factor_1,2-Diethylbenzene_Ratio",Individual Component to Light End Split Factor_n-Undecane_Ratio,Individual Component to Light End Split Factor_n-Pentylbenzene_Ratio,Individual Component to Light End Split Factor_n-Dodecane_Ratio,Individual Component to Light End Split Factor_n-Hexylbenzene_Ratio,Individual Component to Light End Split Factor_n-Tridecane_Ratio,Individual Component to Heart Cut Split Factor_Oxygen_Ratio,Individual Component to Heart Cut Split Factor_Methane_Ratio,Individual Component to Heart Cut Split Factor_Ethane_Ratio,Individual Component to Heart Cut Split Factor_Propane_Ratio,Individual Component to Heart Cut Split Factor_n-Butane_Ratio,Individual Component to Hea

# step4

In [15]:
y

,Light End Product Properties_Oxygen_wt%,Light End Product Properties_Methane_wt%,Light End Product Properties_Ethane_wt%,Light End Product Properties_Propane_wt%,Light End Product Properties_n-Butane_wt%,Light End Product Properties_i-Pentane_wt%,Light End Product Properties_n-Pentane_wt%,Light End Product Properties_tr2-Pentene_wt%,Light End Product Properties_Cyclopentane_wt%,Light End Product Properties_3-Methylpentane_wt%,Light End Product Properties_tr2-Hexene_wt%,Light End Product Properties_n-Hexane_wt%,Light End Product Properties_Methylcyclopentane_wt%,"Light End Product Properties_2,2-Dimethylpentane_wt%",Light End Product Properties_Benzene_wt%,Light End Product Properties_Cyclohexane_wt%,"Light End Product Properties_2,2,3-Trimethylbutane_wt%",Light End Product Properties_4-Methyl-1-hexene_wt%,Light End Product Properties_2-Methylhexane_wt%,"Light End Product Properties_t-1,3-Dimethylcyclopentane_wt%",Light End Product Properties_n-Heptane_wt%,Light End Product Properties_H2O_wt%,Light End Product Properties_Methylcyclohexane_wt%,Light End Product Properties_Ethylcyclopentane_wt%,"Light End Product Properties_2,3-Dimethyl-1-hexene_wt%",Light End Product Properties_Toluene_wt%,Light End Product Properties_2-Methylheptane_wt%,Light End Product Properties_n-Octane_wt%,Light End Product Properties_Ethylcyclohexane_wt%,Light End Product Properties_EthylBenzene_wt%,Light End Product Properties_P-Xylene_wt%,Light End Product Properties_M-Xylene_wt%,Light End Product Properties_O-Xylene_wt%,Light End Product Properties_2-Methyloctane_wt%,Light End Product Properties_1-Nonene_wt%,Light End Product Properties_n-Nonane_wt%,Light End Product Properties_i-Propylbenzene_wt%,"Light End Product Properties_1-trans-3,5-Trimethylcyclohexane_wt%",Light End Product Properties_n-Propylcyclohexane_wt%,Light End Product Properties_n-Propylbenzene_wt%,"Light End Product Properties_1,3,5-Trimethylbenzene_wt%",Light End Product Properties_1-Methyl-2-ethylbenzene_wt%,"Light End Product Properties_1,2,3-Trimethylbenzene_wt%","Light End Product Properties_2,4-Dimethyloctane_wt%","Light End Product Properties_2,2,3,3-Tetramethylhexane_wt%",Light End Product Properties_3-Methylnonane_wt%,Light End Product Properties_tert-Butylcyclohexane_wt%,Light End Product Properties_n-Decane_wt%,"Light End Product Properties_1,2-Diethylbenzene_wt%",Light End Product Properties_n-Undecane_wt%,Light End Product Properties_n-Pentylbenzene_wt%,Light End Product Properties_n-Dodecane_wt%,Light End Product Properties_n-Hexylbenzene_wt%,Light End Product Properties_n-Tridecane_wt%,Heart Cut Product Properties_Oxygen_wt%,Heart Cut Product Properties_Methane_wt%,Heart Cut Product Properties_Ethane_wt%,Heart Cut Product Properties_Propane_wt%,Heart Cut Product Properties_n-Butane_wt%,Heart Cut Product Properties_i-Pentane_wt%,Heart Cut Product Properties_n-Pentane_wt%,Heart Cut Product Properties_tr2-Pentene_wt%,Heart Cut Product Properties_Cyclopentane_wt%,Heart Cut Product Properties_3-Methylpentane_wt%,Heart Cut Product Properties_tr2-Hexene_wt%,Heart Cut Product Properties_n-Hexane_wt%,Heart Cut Product Properties_Methylcyclopentane_wt%,"Heart Cut Product Properties_2,2-Dimethylpentane_wt%",Heart Cut Product Properties_Benzene_wt%,Heart Cut Product Properties_Cyclohexane_wt%,"Heart Cut Product Properties_2,2,3-Trimethylbutane_wt%",Heart Cut Product Properties_4-Methyl-1-hexene_wt%,Heart Cut Product Properties_2-Methylhexane_wt%,"Heart Cut Product Properties_t-1,3-Dimethylcyclopentane_wt%",Heart Cut Product Properties_n-Heptane_wt%,Heart Cut Product Properties_H2O_wt%,Heart Cut Product Properties_Methylcyclohexane_wt%,Heart Cut Product Properties_Ethylcyclopentane_wt%,"Heart Cut Product Properties_2,3-Dimethyl-1-hexene_wt%",Heart Cut Product Properties_Toluene_wt%,Heart Cut Product Properties_2-Methylheptane_wt%,Heart Cut Product Properties_n-Octane_wt%,Heart Cut Product Properties_Ethylcyclohexane_wt%,Heart Cut Product Properties_EthylBenzene_wt%,Heart Cut Product

In [16]:
xna = df[col_names['xna']]

In [17]:
y_pred = D(xna,sp_pred)

In [18]:
y_pred.index = idx
y_pred

,Light End Product Properties_Oxygen_wt%,Light End Product Properties_Methane_wt%,Light End Product Properties_Ethane_wt%,Light End Product Properties_Propane_wt%,Light End Product Properties_n-Butane_wt%,Light End Product Properties_i-Pentane_wt%,Light End Product Properties_n-Pentane_wt%,Light End Product Properties_tr2-Pentene_wt%,Light End Product Properties_Cyclopentane_wt%,Light End Product Properties_3-Methylpentane_wt%,Light End Product Properties_tr2-Hexene_wt%,Light End Product Properties_n-Hexane_wt%,Light End Product Properties_Methylcyclopentane_wt%,"Light End Product Properties_2,2-Dimethylpentane_wt%",Light End Product Properties_Benzene_wt%,Light End Product Properties_Cyclohexane_wt%,"Light End Product Properties_2,2,3-Trimethylbutane_wt%",Light End Product Properties_4-Methyl-1-hexene_wt%,Light End Product Properties_2-Methylhexane_wt%,"Light End Product Properties_t-1,3-Dimethylcyclopentane_wt%",Light End Product Properties_n-Heptane_wt%,Light End Product Properties_H2O_wt%,Light End Product Properties_Methylcyclohexane_wt%,Light End Product Properties_Ethylcyclopentane_wt%,"Light End Product Properties_2,3-Dimethyl-1-hexene_wt%",Light End Product Properties_Toluene_wt%,Light End Product Properties_2-Methylheptane_wt%,Light End Product Properties_n-Octane_wt%,Light End Product Properties_Ethylcyclohexane_wt%,Light End Product Properties_EthylBenzene_wt%,Light End Product Properties_P-Xylene_wt%,Light End Product Properties_M-Xylene_wt%,Light End Product Properties_O-Xylene_wt%,Light End Product Properties_2-Methyloctane_wt%,Light End Product Properties_1-Nonene_wt%,Light End Product Properties_n-Nonane_wt%,Light End Product Properties_i-Propylbenzene_wt%,"Light End Product Properties_1-trans-3,5-Trimethylcyclohexane_wt%",Light End Product Properties_n-Propylcyclohexane_wt%,Light End Product Properties_n-Propylbenzene_wt%,"Light End Product Properties_1,3,5-Trimethylbenzene_wt%",Light End Product Properties_1-Methyl-2-ethylbenzene_wt%,"Light End Product Properties_1,2,3-Trimethylbenzene_wt%","Light End Product Properties_2,4-Dimethyloctane_wt%","Light End Product Properties_2,2,3,3-Tetramethylhexane_wt%",Light End Product Properties_3-Methylnonane_wt%,Light End Product Properties_tert-Butylcyclohexane_wt%,Light End Product Properties_n-Decane_wt%,"Light End Product Properties_1,2-Diethylbenzene_wt%",Light End Product Properties_n-Undecane_wt%,Light End Product Properties_n-Pentylbenzene_wt%,Light End Product Properties_n-Dodecane_wt%,Light End Product Properties_n-Hexylbenzene_wt%,Light End Product Properties_n-Tridecane_wt%,Heart Cut Product Properties_Oxygen_wt%,Heart Cut Product Properties_Methane_wt%,Heart Cut Product Properties_Ethane_wt%,Heart Cut Product Properties_Propane_wt%,Heart Cut Product Properties_n-Butane_wt%,Heart Cut Product Properties_i-Pentane_wt%,Heart Cut Product Properties_n-Pentane_wt%,Heart Cut Product Properties_tr2-Pentene_wt%,Heart Cut Product Properties_Cyclopentane_wt%,Heart Cut Product Properties_3-Methylpentane_wt%,Heart Cut Product Properties_tr2-Hexene_wt%,Heart Cut Product Properties_n-Hexane_wt%,Heart Cut Product Properties_Methylcyclopentane_wt%,"Heart Cut Product Properties_2,2-Dimethylpentane_wt%",Heart Cut Product Properties_Benzene_wt%,Heart Cut Product Properties_Cyclohexane_wt%,"Heart Cut Product Properties_2,2,3-Trimethylbutane_wt%",Heart Cut Product Properties_4-Methyl-1-hexene_wt%,Heart Cut Product Properties_2-Methylhexane_wt%,"Heart Cut Product Properties_t-1,3-Dimethylcyclopentane_wt%",Heart Cut Product Properties_n-Heptane_wt%,Heart Cut Product Properties_H2O_wt%,Heart Cut Product Properties_Methylcyclohexane_wt%,Heart Cut Product Properties_Ethylcyclopentane_wt%,"Heart Cut Product Properties_2,3-Dimethyl-1-hexene_wt%",Heart Cut Product Properties_Toluene_wt%,Heart Cut Product Properties_2-Methylheptane_wt%,Heart Cut Product Properties_n-Octane_wt%,Heart Cut Product Properties_Ethylcyclohexane_wt%,Heart Cut Product Properties_EthylBenzene_wt%,Heart Cut Product

# mse

In [19]:
mse = np.mean((y-y_pred)**2,axis=1)
mse.head()

017-01    0.001738
017-02    0.002129
017-03    0.003613
017-04    0.002605
017-05    0.002154
dtype: float64

In [20]:
print(mse.mean())

0.04150129327545787


# save

In [21]:
data = pd.DataFrame()
for i,j in zip(y.index,y_pred.index):
    row1 = y.loc[[i]]
    row2 = y_pred.loc[[i]]
    data = data.append(row1.append(row2))

In [22]:
data = data.apply(lambda x:round(x,3))
data

,Light End Product Properties_Oxygen_wt%,Light End Product Properties_Methane_wt%,Light End Product Properties_Ethane_wt%,Light End Product Properties_Propane_wt%,Light End Product Properties_n-Butane_wt%,Light End Product Properties_i-Pentane_wt%,Light End Product Properties_n-Pentane_wt%,Light End Product Properties_tr2-Pentene_wt%,Light End Product Properties_Cyclopentane_wt%,Light End Product Properties_3-Methylpentane_wt%,Light End Product Properties_tr2-Hexene_wt%,Light End Product Properties_n-Hexane_wt%,Light End Product Properties_Methylcyclopentane_wt%,"Light End Product Properties_2,2-Dimethylpentane_wt%",Light End Product Properties_Benzene_wt%,Light End Product Properties_Cyclohexane_wt%,"Light End Product Properties_2,2,3-Trimethylbutane_wt%",Light End Product Properties_4-Methyl-1-hexene_wt%,Light End Product Properties_2-Methylhexane_wt%,"Light End Product Properties_t-1,3-Dimethylcyclopentane_wt%",Light End Product Properties_n-Heptane_wt%,Light End Product Properties_H2O_wt%,Light End Product Properties_Methylcyclohexane_wt%,Light End Product Properties_Ethylcyclopentane_wt%,"Light End Product Properties_2,3-Dimethyl-1-hexene_wt%",Light End Product Properties_Toluene_wt%,Light End Product Properties_2-Methylheptane_wt%,Light End Product Properties_n-Octane_wt%,Light End Product Properties_Ethylcyclohexane_wt%,Light End Product Properties_EthylBenzene_wt%,Light End Product Properties_P-Xylene_wt%,Light End Product Properties_M-Xylene_wt%,Light End Product Properties_O-Xylene_wt%,Light End Product Properties_2-Methyloctane_wt%,Light End Product Properties_1-Nonene_wt%,Light End Product Properties_n-Nonane_wt%,Light End Product Properties_i-Propylbenzene_wt%,"Light End Product Properties_1-trans-3,5-Trimethylcyclohexane_wt%",Light End Product Properties_n-Propylcyclohexane_wt%,Light End Product Properties_n-Propylbenzene_wt%,"Light End Product Properties_1,3,5-Trimethylbenzene_wt%",Light End Product Properties_1-Methyl-2-ethylbenzene_wt%,"Light End Product Properties_1,2,3-Trimethylbenzene_wt%","Light End Product Properties_2,4-Dimethyloctane_wt%","Light End Product Properties_2,2,3,3-Tetramethylhexane_wt%",Light End Product Properties_3-Methylnonane_wt%,Light End Product Properties_tert-Butylcyclohexane_wt%,Light End Product Properties_n-Decane_wt%,"Light End Product Properties_1,2-Diethylbenzene_wt%",Light End Product Properties_n-Undecane_wt%,Light End Product Properties_n-Pentylbenzene_wt%,Light End Product Properties_n-Dodecane_wt%,Light End Product Properties_n-Hexylbenzene_wt%,Light End Product Properties_n-Tridecane_wt%,Heart Cut Product Properties_Oxygen_wt%,Heart Cut Product Properties_Methane_wt%,Heart Cut Product Properties_Ethane_wt%,Heart Cut Product Properties_Propane_wt%,Heart Cut Product Properties_n-Butane_wt%,Heart Cut Product Properties_i-Pentane_wt%,Heart Cut Product Properties_n-Pentane_wt%,Heart Cut Product Properties_tr2-Pentene_wt%,Heart Cut Product Properties_Cyclopentane_wt%,Heart Cut Product Properties_3-Methylpentane_wt%,Heart Cut Product Properties_tr2-Hexene_wt%,Heart Cut Product Properties_n-Hexane_wt%,Heart Cut Product Properties_Methylcyclopentane_wt%,"Heart Cut Product Properties_2,2-Dimethylpentane_wt%",Heart Cut Product Properties_Benzene_wt%,Heart Cut Product Properties_Cyclohexane_wt%,"Heart Cut Product Properties_2,2,3-Trimethylbutane_wt%",Heart Cut Product Properties_4-Methyl-1-hexene_wt%,Heart Cut Product Properties_2-Methylhexane_wt%,"Heart Cut Product Properties_t-1,3-Dimethylcyclopentane_wt%",Heart Cut Product Properties_n-Heptane_wt%,Heart Cut Product Properties_H2O_wt%,Heart Cut Product Properties_Methylcyclohexane_wt%,Heart Cut Product Properties_Ethylcyclopentane_wt%,"Heart Cut Product Properties_2,3-Dimethyl-1-hexene_wt%",Heart Cut Product Properties_Toluene_wt%,Heart Cut Product Properties_2-Methylheptane_wt%,Heart Cut Product Properties_n-Octane_wt%,Heart Cut Product Properties_Ethylcyclohexane_wt%,Heart Cut Product Properties_EthylBenzene_wt%,Heart Cut Product

In [23]:
case = df[col_names['case']]
data = case.join(data)
data.head()

,Case Conditions_Feed Rate to C013 (Input)_m3/hr,Case Conditions_Heart Cut Prod. Rate (Input)_m3/hr,Case Conditions_C7+ in Light End (Input)_wt%,Case Conditions_C6P- in Heart Cut (Input)_wt%,Light End Product Properties_Oxygen_wt%,Light End Product Properties_Methane_wt%,Light End Product Properties_Ethane_wt%,Light End Product Properties_Propane_wt%,Light End Product Properties_n-Butane_wt%,Light End Product Properties_i-Pentane_wt%,Light End Product Properties_n-Pentane_wt%,Light End Product Properties_tr2-Pentene_wt%,Light End Product Properties_Cyclopentane_wt%,Light End Product Properties_3-Methylpentane_wt%,Light End Product Properties_tr2-Hexene_wt%,Light End Product Properties_n-Hexane_wt%,Light End Product Properties_Methylcyclopentane_wt%,"Light End Product Properties_2,2-Dimethylpentane_wt%",Light End Product Properties_Benzene_wt%,Light End Product Properties_Cyclohexane_wt%,"Light End Product Properties_2,2,3-Trimethylbutane_wt%",Light End Product Properties_4-Methyl-1-hexene_wt%,Light End Product Properties_2-Methylhexane_wt%,"Light End Product Properties_t-1,3-Dimethylcyclopentane_wt%",Light End Product Properties_n-Heptane_wt%,Light End Product Properties_H2O_wt%,Light End Product Properties_Methylcyclohexane_wt%,Light End Product Properties_Ethylcyclopentane_wt%,"Light End Product Properties_2,3-Dimethyl-1-hexene_wt%",Light End Product Properties_Toluene_wt%,Light End Product Properties_2-Methylheptane_wt%,Light End Product Properties_n-Octane_wt%,Light End Product Properties_Ethylcyclohexane_wt%,Light End Product Properties_EthylBenzene_wt%,Light End Product Properties_P-Xylene_wt%,Light End Product Properties_M-Xylene_wt%,Light End Product Properties_O-Xylene_wt%,Light End Product Properties_2-Methyloctane_wt%,Light End Product Properties_1-Nonene_wt%,Light End Product Properties_n-Nonane_wt%,Light End Product Properties_i-Propylbenzene_wt%,"Light End Product Properties_1-trans-3,5-Trimethylcyclohexane_wt%",Light End Product Properties_n-Propylcyclohexane_wt%,Light End Product Properties_n-Propylbenzene_wt%,"Light End Product Properties_1,3,5-Trimethylbenzene_wt%",Light End Product Properties_1-Methyl-2-ethylbenzene_wt%,"Light End Product Properties_1,2,3-Trimethylbenzene_wt%","Light End Product Properties_2,4-Dimethyloctane_wt%","Light End Product Properties_2,2,3,3-Tetramethylhexane_wt%",Light End Product Properties_3-Methylnonane_wt%,Light End Product Properties_tert-Butylcyclohexane_wt%,Light End Product Properties_n-Decane_wt%,"Light End Product Properties_1,2-Diethylbenzene_wt%",Light End Product Properties_n-Undecane_wt%,Light End Product Properties_n-Pentylbenzene_wt%,Light End Product Properties_n-Dodecane_wt%,Light End Product Properties_n-Hexylbenzene_wt%,Light End Product Properties_n-Tridecane_wt%,Heart Cut Product Properties_Oxygen_wt%,Heart Cut Product Properties_Methane_wt%,Heart Cut Product Properties_Ethane_wt%,Heart Cut Product Properties_Propane_wt%,Heart Cut Product Properties_n-Butane_wt%,Heart Cut Product Properties_i-Pentane_wt%,Heart Cut Product Properties_n-Pentane_wt%,Heart Cut Product Properties_tr2-Pentene_wt%,Heart Cut Product Properties_Cyclopentane_wt%,Heart Cut Product Properties_3-Methylpentane_wt%,Heart Cut Product Properties_tr2-Hexene_wt%,Heart Cut Product Properties_n-Hexane_wt%,Heart Cut Product Properties_Methylcyclopentane_wt%,"Heart Cut Product Properties_2,2-Dimethylpentane_wt%",Heart Cut Product Properties_Benzene_wt%,Heart Cut Product Properties_Cyclohexane_wt%,"Heart Cut Product Properties_2,2,3-Trimethylbutane_wt%",Heart Cut Product Properties_4-Methyl-1-hexene_wt%,Heart Cut Product Properties_2-Methylhexane_wt%,"Heart Cut Product Properties_t-1,3-Dimethylcyclopentane_wt%",Heart Cut Product Properties_n-Heptane_wt%,Heart Cut Product Properties_H2O_wt%,Heart Cut Product Properties_Methylcyclohexane_wt%,Heart Cut Product Properties_Ethylcyclopentane_wt%,"Heart Cut Product Properties_2,3-Dimethyl-1-hexene_wt%",Heart Cut Product Properties_Toluene_wt%,Heart Cut Prod

In [24]:
#data.to_excel('./預測結果(偶數行為預測).xlsx')

# step5

In [25]:
df = pd.read_csv('./data/phase_2/cleaned/train(energy).csv',index_col=0).dropna(axis=0)
df.head()

,Case Conditions_Feed Rate to C013 (Input)_m3/hr,Case Conditions_Heart Cut Prod. Rate (Input)_m3/hr,Case Conditions_C7+ in Light End (Input)_wt%,Case Conditions_C6P- in Heart Cut (Input)_wt%,Naphtha Properties_Oxygen_wt%,Naphtha Properties_Methane_wt%,Naphtha Properties_Ethane_wt%,Naphtha Properties_Propane_wt%,Naphtha Properties_n-Butane_wt%,Naphtha Properties_i-Pentane_wt%,Naphtha Properties_n-Pentane_wt%,Naphtha Properties_tr2-Pentene_wt%,Naphtha Properties_Cyclopentane_wt%,Naphtha Properties_3-Methylpentane_wt%,Naphtha Properties_tr2-Hexene_wt%,Naphtha Properties_n-Hexane_wt%,Naphtha Properties_Methylcyclopentane_wt%,"Naphtha Properties_2,2-Dimethylpentane_wt%",Naphtha Properties_Benzene_wt%,Naphtha Properties_Cyclohexane_wt%,"Naphtha Properties_2,2,3-Trimethylbutane_wt%",Naphtha Properties_4-Methyl-1-hexene_wt%,Naphtha Properties_2-Methylhexane_wt%,"Naphtha Properties_t-1,3-Dimethylcyclopentane_wt%",Naphtha Properties_n-Heptane_wt%,Naphtha Properties_H2O_wt%,Naphtha Properties_Methylcyclohexane_wt%,Naphtha Properties_Ethylcyclopentane_wt%,"Naphtha Properties_2,3-Dimethyl-1-hexene_wt%",Naphtha Properties_Toluene_wt%,Naphtha Properties_2-Methylheptane_wt%,Naphtha Properties_n-Octane_wt%,Naphtha Properties_Ethylcyclohexane_wt%,Naphtha Properties_EthylBenzene_wt%,Naphtha Properties_P-Xylene_wt%,Naphtha Properties_M-Xylene_wt%,Naphtha Properties_O-Xylene_wt%,Naphtha Properties_2-Methyloctane_wt%,Naphtha Properties_1-Nonene_wt%,Naphtha Properties_n-Nonane_wt%,Naphtha Properties_i-Propylbenzene_wt%,"Naphtha Properties_1-trans-3,5-Trimethylcyclohexane_wt%",Naphtha Properties_n-Propylcyclohexane_wt%,Naphtha Properties_n-Propylbenzene_wt%,"Naphtha Properties_1,3,5-Trimethylbenzene_wt%",Naphtha Properties_1-Methyl-2-ethylbenzene_wt%,"Naphtha Properties_1,2,3-Trimethylbenzene_wt%","Naphtha Properties_2,4-Dimethyloctane_wt%","Naphtha Properties_2,2,3,3-Tetramethylhexane_wt%",Naphtha Properties_3-Methylnonane_wt%,Naphtha Properties_tert-Butylcyclohexane_wt%,Naphtha Properties_n-Decane_wt%,"Naphtha Properties_1,2-Diethylbenzene_wt%",Naphtha Properties_n-Undecane_wt%,Naphtha Properties_n-Pentylbenzene_wt%,Naphtha Properties_n-Dodecane_wt%,Naphtha Properties_n-Hexylbenzene_wt%,Naphtha Properties_n-Tridecane_wt%,Duty_E062 3.5K Steam Rate_ton/hr,Duty_Condenser Heat Duty_Mkcal/hr,Duty_Reboiler Heat Duty_Mkcal/hr,Naphtha Properties_Naphtha Feed Rate_m3/hr,Light End Product Properties_Light End Prod. Rate_m3/hr,Heart Cut Product Properties_Heart Cut Prod. Rate_m3/hr,Heavy End Product Properties_Heavy End Product Rate_m3/hr,Naphtha Properties_Naphtha Feed Rate_ton/hr,Light End Product Properties_Light End Prod. Rate_ton/hr,Heart Cut Product Properties_Heart Cut Prod. Rate_ton/hr,Heavy End Product Properties_Heavy End Product Rate_ton/hr
001-01,10.0,100.0,2.5,1.0,0.0,0.0,0.0,0.002,0.003,0.025999,0.073999,0.001,0.013,1.365973,0.027999,4.270915,1.50397,0.742985,0.403992,0.813984,0.815984,0.016,6.178877,0.551989,10.797785,0.0,3.195936,2.212956,0.110998,2.855943,10.991779,10.389793,5.176897,1.01798,0.577989,1.425972,1.266975,10.083797,0.262995,6.692866,0.047999,2.640948,1.760965,0.204996,0.878982,1.299974,0.954981,0.935981,1.387972,3.239935,0.574988,1.168977,0.753985,0.220996,0.043999,0.004,0.008,0.0,2.280675,3.891577,4.252277,118.000084,7.997900,100.000198,10.001979,86.475102,5.469008,73.401711,7.604384
001-02,10.0,100.0,5.0,1.0,0.0,0.0,0.0,0.002,0.003,0.025999,0.073999,0.001,0.013,1.365973,0.027999,4.270915,1.50397,0.742985,0.403992,0.813984,0.815984,0.016,6.178877,0.551989,10.797785,0.0,3.195936,2.212956,0.110998,2.855943,10.991779,10.389793,5.176897,1.01798,0.577989,1.425972,1.266975,10.083797,0.262995,6.692866,0.047999,2.640948,1.760965,0.204996,0.878982,1.299974,0.954981,0.935981,1.387972,3.239935,0.574988,1.168977,0.753985,0.220996,0.043999,0.004,0.008,0.0,2.280843,3.723144,4.071568,118.000000,8.445992,100.000122,9.553886,86.475047,5.797086,73.411102,7.266858
001-03,10.0,100.0,10.0,1.0,0.0,0.0,0.0,0.002,0.003,0.025999,0.073999,0.001,0.013,1.3

In [26]:
duty = df[col_names['duty']].drop('Duty_Condenser Heat Duty_Mkcal/hr',axis=1)
density = df[col_names['Rate_ton']].values / df[col_names['Rate_m3']].values 
density = pd.DataFrame(density,columns = E.y_col[-4:],index=df.index)
y_real = duty.join(density)
y_real.head()

,Duty_E062 3.5K Steam Rate_ton/hr,Duty_Reboiler Heat Duty_Mkcal/hr,NA_density,LE_density,HC_density,HE_density
001-01,2.280675,4.252277,0.732839,0.683806,0.734016,0.760288
001-02,2.280843,4.071568,0.732839,0.686371,0.734110,0.760618
001-03,2.280798,3.900158,0.732839,0.688468,0.734355,0.761531
001-04,2.280843,3.759819,0.732839,0.683457,0.733602,0.759290
001-05,2.280675,3.607212,0.732839,0.686087,0.733705,0.759360


In [27]:
y_pred = E.predict(df[E.x_col])

In [28]:
y_pred.index = df.index 
y_pred.head()

,Duty_E062 3.5K Steam Rate_ton/hr,Duty_Reboiler Heat Duty_Mkcal/hr,NA_density,LE_density,HC_density,HE_density
001-01,2.270860,4.329087,0.735319,0.680120,0.732836,0.764586
001-02,2.272807,4.097394,0.737237,0.686888,0.732755,0.765968
001-03,2.267768,3.858021,0.738942,0.692674,0.731816,0.768321
001-04,2.261580,3.767885,0.735107,0.679319,0.731006,0.762847
001-05,2.265015,3.593066,0.737056,0.685421,0.730973,0.764941
